## Run Object Detection

In [1]:
class_names = ["BG"]
class_names.append("Rt Upper CB")

Epochs = 40
ImageSize = 1024 
Batch = 1 
DatasetFolder = "Generated_Data_20180410_191400_Seg_Base_Expand_20pixel_Cropped"
DrawImg = False

FinalModelFile = "MaskRCNN_Rt Upper CB_20180410_202127.hdf5"

import socket 

GPU_ServerType = "Local_83"

hostName = socket.gethostname()
if(hostName == "deepserver"):
    GPU_ServerType = "Local_83"
elif (hostName[:3] == "csi"):
    GPU_ServerType = "Kakao"
elif (hostName == "user-All-Series"):
    GPU_ServerType = "Local_180"    
elif (hostName == "User-PC"):
    GPU_ServerType = "Local_PC" 
    
import os
import XRay_DataSet
import model as modellib
from datetime import datetime
import csv

BaseDataSetPath = ""
GPU_Num = ""

if(GPU_ServerType == "Local_83"):
    BaseDataSetPath = "/data/gyungchan2110/Segmentation"
    GPU_Num = "3"
elif (GPU_ServerType == "Kakao"):
    BaseDataSetPath = "/data/private/cardiomegaly"
    GPU_Num = "0"
elif (GPU_ServerType == "Local_180"):
    BaseDataSetPath = ""
    GPU_Num = "0"
elif (GPU_ServerType == "Local_PC"):
    BaseDataSetPath = ""
    GPU_Num = "0"

os.environ["CUDA_VISIBLE_DEVICES"]= GPU_Num
ROOT_DIR = os.getcwd()
LOG_DIR = ROOT_DIR + "/logs"
MODEL_DIR = ROOT_DIR + "/Models"
FinalModelPath = MODEL_DIR + "/" + FinalModelFile
##########################################################################################################################

config = XRay_DataSet.Xray_CardioMegalyModelConfig(len(class_names) - 1, ImageSize, Batch)
config.display()

model = modellib.MaskRCNN(mode="inference", model_dir=LOG_DIR, config=config)
model.load_weights(FinalModelPath, by_name=True)


import matplotlib.pyplot as plt

def Dice(y_true, y_pred):
    """Returns Dice Similarity Coefficient for ground truth and predicted masks."""
    length = len(y_true)
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    #print(y_pred.shape, y_true.shape)
    assert y_true.shape == y_pred.shape

    intersection = np.logical_and(y_true, y_pred)  
    true_sum = np.zeros(length)
    pred_sum = np.zeros(length)
    intersection_sum = np.zeros(length)
    for i in range(length):
        true_sum[i]= y_true[i,:,:].sum()
        pred_sum[i]= y_pred[i,:,:].sum()
        intersection_sum[i] = intersection[i,:,:].sum()

    dices = np.zeros(length)

    #for i in range(length):
    dices[:] = (2 * intersection_sum[:] + 1.) / (true_sum[:] + pred_sum[:] + 1.)

    return dices

def estimate_IoU(true_Loc, pred_Loc):
    
#     if(not operator.eq(true_Loc[0], pred_Loc[0])):
#         return

    true_x_start = true_Loc[0]
    true_y_start = true_Loc[1]
    true_x_end = true_x_start + true_Loc[2]
    true_y_end = true_y_start + true_Loc[3]

    pred_x_start = pred_Loc[0]
    pred_y_start = pred_Loc[1]
    pred_x_end = pred_x_start + pred_Loc[2]
    pred_y_end = pred_y_start + pred_Loc[3]

    start_x = min(true_x_start, pred_x_start)
    start_y = min(true_y_start, pred_y_start)
    end_x = max(true_x_end, pred_x_end)
    end_y = max(true_y_end, pred_y_end)

    true_label = np.zeros((end_x - start_x, end_y - start_y))
    pred_label = np.zeros((end_x - start_x, end_y - start_y))

    true_label[(true_x_start -start_x) : (true_x_end - start_x), (true_y_start - start_y):(true_y_end - start_y) ] = 1
    pred_label[(pred_x_start -start_x) : (pred_x_end - start_x), (pred_y_start - start_y):(pred_y_end - start_y) ] = 1

    smooth = 1.
    y_true_f = true_label.flatten()
    y_pred_f = pred_label.flatten()
    intersection = np.logical_and(y_true_f, y_pred_f).sum()
    union = np.logical_or(y_true_f, y_pred_f).sum()
    jac = (intersection) / (union )
    return jac

def Jaccard_coef(y_true, y_pred):
    smooth = 1.
    length = len(y_true)
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    assert  y_true.shape == y_pred.shape

    intersection = np.logical_and(y_true, y_pred)  
    union = np.logical_or(y_true, y_pred) 
    union_sum = np.zeros(length)
    intersection_sum = np.zeros(length)
    for i in range(length):
        union_sum[i]= union[:,:].sum()
        intersection_sum[i] = intersection[i,:,:].sum()

    jacards = np.zeros(length)

    #for i in range(length):
    jacards[:] = (intersection_sum[:] + smooth) / (union_sum[:] + smooth)

    return jacards 


def GetMostProperMask(results, truemask, dstPath, filename):
    scores = results['scores']
    masks =  results['masks']
    bboxes = results['rois']
    
    if(len(scores) == 0):
        return False, 0,0,0
    scores = np.asarray(scores)
    maxvalue = np.amax(scores)
    
    indece = []
    for i, score in enumerate(scores) : 
        if (score == maxvalue):
            indece.append(i)
    
    indece = np.asarray(indece) 
    
    indece_tmp = []
    index = -1 
    tmp = 1024
    if(len(indece) == 1):
        index = indece[0] 
    else:
        for i in indece : 
            y1, x1, y2, x2,_ = bboxes[i]
            if(x < 512 and y1 < tmp):
                tmp = y1 
                indece_tmp.append(i)
    
    tmp = 0
    if(len(indece_tmp) == 1):
        index = indece_tmp[0] 
    else:
        for i in indece_tmp : 
            y1, x1, y2, x2,_ = bboxes[i]
            if(x1 >  tmp):
                tmp = x1 
                index = i
    print("index", index)
       #################################################### 
    truemask = cv2.imread(MaskDIR + "/" + filename, 0)
    truemask = np.asarray(truemask, dtype = "uint8")
    
    _, thresh = cv2.threshold(truemask, 127, 255, cv2.THRESH_BINARY)
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE )
    rect = cv2.boundingRect(contours[0]) # y, x , h, w 
    #print("rect : ", rect)
    box = bboxes[index] # y1,x1 y2,x2
    #print("box : ", box)
    iou = estimate_IoU(rect, [box[1], box[0], box[3] - box[1], box[2] - box[0]])
    #################################################################
    truemask = truemask / 255
    #for index in indeces : 
    mask = masks[:, :, index]
    mask = np.asarray(mask)
    mask = np.expand_dims(mask, 0)
    
    truemask = np.asarray(truemask)
    truemask = np.expand_dims(truemask, 0)
    
    dice = Dice(truemask, mask)
    jaccard = Jaccard_coef(truemask, mask)
    print("dice : ", dice, "Jaccard : ", jaccard, "IoU : ", iou)
    
    truemask = truemask * 255
    mask = mask * 255
    if(DrawImg):
        cv2.imwrite(dstPath + "/" + filename, mask[0,:,:])
        #cv2.imwrite("/data/gyungchan2110/tmp_orig.png", truemask[0,:,:])
        fig = plt.figure()
        fig.set_size_inches(256/256, 1, forward=False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)


        plt.imshow(truemask[0,:,:], cmap = "gray")
        ax.imshow(mask[0,:,:], cmap='gray', alpha = 0.15, interpolation = 'nearest')
        #plt.show()
        fig.savefig(dstPath + "/" + filename[:-4] + "_Overlay.png",dpi = 1024)
    return True, dice, jaccard, iou

import cv2
import visualize
import numpy as np
import random

IMAGE_DIR = BaseDataSetPath + "/" + DatasetFolder + "/Imgs/test"
MaskDIR = BaseDataSetPath + "/" + DatasetFolder + "/Masks/" + class_names[1] + "/test"
dstPath = BaseDataSetPath + "/TestResult"
file_names = os.listdir(IMAGE_DIR)
dices = []
jaccards = []
ious = []
finals = []
#filename = random.choice(file_names)
icount = 0
for filename in file_names:
    
    if DrawImg : 
        filename = random.choice(file_names)
        
    image = cv2.imread(os.path.join(IMAGE_DIR, filename))

    truemask = cv2.imread(MaskDIR + "/" + filename, 0)
    truemask = np.asarray(truemask, dtype = "uint8")



    image = np.asarray(image)

    #image = np.expand_dims(image,-1)
    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results
    r = results[0]
    print(filename)
    result, dice, jaccard,iou = GetMostProperMask(r, truemask, dstPath, filename)
    if(DrawImg ):
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'], dstPath = dstPath, filename = filename,truemask = truemask)
        
        datas = [filename, dice, jaccard, iou]
        f = open(dstPath + "/TestResult.csv", 'a')
        f_writer = csv.writer(f)
        f_writer.writerow(datas)
        f.close()
    finals.append(result)
    if result:
        dices.append(dice)
        jaccards.append(jaccard)
        ious.append(iou)
    icount += 1
    if DrawImg and icount >= 10: 
        break
    
dices = np.asarray(dices)
jaccards = np.asarray(jaccards) 
ious = np.asarray(ious) 
print("Dice : " , dices.mean(), dices.std())

print("Jaccard : " , jaccards.mean(), jaccards.std())

print("IoU : " , ious.mean(), ious.std())

print(finals)
#

/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.



Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
ConfigType                     Model
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EPOCHS                         50
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  1024
IMAGE_PADDING                  False
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           Cardiomegaly
NUM_CLASSES                    2
POOL_SIZE       

In [2]:
dices = np.asarray(dices)
jaccards = np.asarray(jaccards) 
ious = np.asarray(ious) 
print(dices.mean(), jaccards.mean(), ious.mean())
print(results)

0.7237618416468002 0.5531427413320728 0.6379893247802448
[{'scores': array([0.81280863, 0.73259854], dtype=float32), 'masks': array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=uint8), 'rois': array([[287, 657, 551, 721],
       [367, 315, 654, 389]], dtype=int32), 'class_ids': array([1, 1], dtype=int32)}]


In [3]:
#ious_ = np.nonzero(ious)
suma = 0.0
count = 0 
nonzero = []
for data in ious : 
    if(data != 0.0) :
        nonzero.append(data)
nonzero = np.asarray(nonzero)
print(nonzero.mean(), nonzero.std())


0.6427035316135963 0.14737962724619819


In [4]:
print(len(nonzero) / len(ious))

0.9926650366748166
